# <font color='orange'>TME IA ET ROBOTIQUE (M2 Androide, SU)</font>
# <font color='orange'>Swarm Robotics, part 1 : collective motion</font>

# <font color="green">Version ETUDIANT·E·S 2025-2026</font>

*mise à jour: 18/12/2025*

Ce notebook doit être exécuté dans [Google Colab](colab.research.google.com/) ou dans Jupyter.

A la fin du TME, vous devez déposer votre travail sur Moodle:
* déposer votre notebook, avec le nom de fichier *obligatoirement* au format suivant: **IAR_NOM1_NOM2.ipynb**
* toutes les cellules doivent être exécutées avec les graphes visibles
* un **bref** commentaire lorsque c'est demandé. Pour toutes les questions, une à deux phrases suffisent

*Le sujet est à faire en binome.*


---
---
---

# <font color='orange'>PREAMBULE: initialisation du simulateur de Boids</font>


Le code definit dans la cellule ci-après permet de simuler le modèle de comportement collectif de type Boids. Dans ce modèle, chaque agent...
- a une vélocité, une direction et un zone de perception limitée au voisinage proche.
- est représenté par une empreinte circulaire qui correspond à une particule dure, c'est à dire que les collisions sont possibles.

Chaque agent suit trois règles de comportements:
1. **Répulsion** : un agent évite les autres agents si la distance qui le sépare de son plus proche voisin est « trop » faible (seuil est à définir).
2. **Attraction** : un agent se rapproche de ses voisins si la distance qui le sépare de son plus proche voisin est « trop » grande (seuil est à définir)
3. **Alignement** : un agent s'oriente pour aller dans la même direction que les particules voisines. C'est le comportement par défaut lorsque le voisin le plus proche n'est ni trop proche ni trop éloigné.

Le simulateur représente les agents comme un cercle avec une flèche, montrant sa direction. La flèche devient rouge en cas de collision. Les bords de l'arène sont à conditions périodiques (i.e. un agent qui sort à droite réapparait à gauche).

<font color='red'>Ce TP ne nécessite pas d'étudiez le code du simulateur</font>. Il est conseillé d'exécuter la cellule suivante sans l'ouvrir (sauf si vous êtes curieux·se, mais svp attendez la fin de la séance). Vous pouvez consulter le cours pour un rappel des règles et des dynamiques typiques des Boids (ainsi que l'étude qu'en a fait Couzin et al. en 2002).


In [1]:
import numpy as np
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 200  # limit to 100+ MB (or higher if needed) for rendering
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
from scipy.spatial import cKDTree
import time

def run_simulation(
    num_particles=64,
    max_steps=200,
    particle_diameter=0.02,
    sensing_radius=8 * 0.02,  # 8 * particle_diameter
    repulsion_threshold=0.3,
    attraction_threshold=0.8,
    speed=0.01,
    rotation_speed=np.pi / 8,
    box_size=1.0,
    rotational_noise_std=0.05,
    enable_collision=True,
    render_animation=True
):


    # Display parameters
    print(f"Simulation Parameters:")
    print(f"Number of particles: {num_particles}")
    print(f"Max steps: {max_steps}")
    print(f"Particle diameter: {particle_diameter}")
    print(f"Sensing radius: {sensing_radius}")
    print(f"Repulsion threshold (normalized): {repulsion_threshold}")
    print(f"Attraction threshold (normalized): {attraction_threshold}")
    print(f"Max speed: {speed}")
    print(f"Max rotation speed: {rotation_speed:.2f}")
    print(f"Rotational noise std: {rotational_noise_std}")
    print(f"Collision enabled: {enable_collision}")
    print(f"Box size: {box_size}")

    # Initialize particles
    np.random.seed(int(time.time()))  # For different initial conditions
    positions = np.random.rand(num_particles, 2) * box_size
    orientations = np.random.uniform(-np.pi, np.pi, num_particles)
    move_flags = np.ones(num_particles, dtype=bool)
    collided = np.zeros(num_particles, dtype=bool)

    # Precompute constants
    half_box_size = 0.5 * box_size

    # Function to apply periodic boundary conditions
    def apply_periodic(pos):
        return pos % box_size

    if render_animation:
        mpl.rcParams['animation.embed_limit'] = 200
        fig, ax = plt.subplots(figsize=(6,6))
        ax.set_xlim(0, box_size)
        ax.set_ylim(0, box_size)
        ax.set_facecolor('black')
        ax.set_aspect('equal')
        plt.axis('off')

        # Prepare data for animation
        def init():
            ax.set_xlim(0, box_size)
            ax.set_ylim(0, box_size)
            ax.set_facecolor('black')
            ax.set_aspect('equal')
            plt.axis('off')
            return []

    # Main simulation loop
    def animate(step):
        nonlocal positions, orientations, move_flags, collided
        if render_animation:
            ax.clear()
            ax.set_xlim(0, box_size)
            ax.set_ylim(0, box_size)
            ax.set_facecolor('black')
            ax.set_aspect('equal')
            plt.axis('off')

        if step % (max_steps // 10) == 0:
            print(f"{int(100 * step / max_steps)}%", end=" ")
        elif step == max_steps - 1:
            print("100% [done].")

        # Reset collision flags
        move_flags[:] = True
        collided[:] = False

        # Build KDTree for neighbor search
        tree = cKDTree(positions, boxsize=box_size)

        # Compute desired new positions and orientations
        desired_positions = positions.copy()
        new_orientations = orientations.copy()
        for i in range(num_particles):
            # Find neighbors within sensing radius
            idxs = tree.query_ball_point(positions[i], sensing_radius)

            # Remove self from neighbor list
            idxs = [j for j in idxs if j != i]

            if idxs:
                # Compute distances to neighbors
                neighbors_pos = positions[idxs]
                delta_pos = neighbors_pos - positions[i]
                # Apply periodic boundary conditions
                delta_pos = (delta_pos + half_box_size) % box_size - half_box_size
                distances = np.hypot(delta_pos[:,0], delta_pos[:,1])

                # Find closest neighbor
                closest_idx = np.argmin(distances)
                closest_dist = distances[closest_idx]
                closest_particle_idx = idxs[closest_idx]

                if closest_dist / sensing_radius < repulsion_threshold:
                    # Repulsion: move away from closest neighbor
                    vec_to_neighbor = delta_pos[closest_idx]
                    angle_away = np.arctan2(-vec_to_neighbor[1], -vec_to_neighbor[0])
                    delta_theta = angle_away - orientations[i]
                elif closest_dist / sensing_radius > attraction_threshold:
                    # Attraction: move towards average position of neighbors
                    avg_position = np.mean(neighbors_pos, axis=0)
                    vec_to_avg = avg_position - positions[i]
                    vec_to_avg = (vec_to_avg + half_box_size) % box_size - half_box_size
                    angle_to_avg = np.arctan2(vec_to_avg[1], vec_to_avg[0])
                    delta_theta = angle_to_avg - orientations[i]
                else:
                    # Alignment: align with average orientation of neighbors
                    neighbor_orientations = orientations[idxs]
                    sin_sum = np.sum(np.sin(neighbor_orientations))
                    cos_sum = np.sum(np.cos(neighbor_orientations))
                    avg_orientation = np.arctan2(sin_sum, cos_sum)
                    delta_theta = avg_orientation - orientations[i]

                # Normalize delta_theta to be between -pi and pi
                delta_theta = (delta_theta + np.pi) % (2 * np.pi) - np.pi
                # Limit delta_theta to rotation_speed
                delta_theta = np.clip(delta_theta, -rotation_speed, rotation_speed)
                new_orientations[i] += delta_theta
                # Add rotational noise
                new_orientations[i] += np.random.normal(0, rotational_noise_std)
                # Keep orientation within [-pi, pi]
                new_orientations[i] = (new_orientations[i] + np.pi) % (2 * np.pi) - np.pi

            # Compute desired move
            dx = speed * np.cos(new_orientations[i])
            dy = speed * np.sin(new_orientations[i])
            desired_positions[i] = positions[i] + np.array([dx, dy])
            # Apply periodic boundary conditions
            desired_positions[i] = desired_positions[i] % box_size

        # Collision detection
        if enable_collision:
            tree_desired = cKDTree(desired_positions, boxsize=box_size)
            pairs = tree_desired.query_pairs(r=particle_diameter)
            pairs = list(pairs)
            if pairs:
                pairs = np.array(pairs)
                collided_indices = np.unique(pairs.flatten())
                move_flags[collided_indices] = False
                collided[collided_indices] = True
        else:
            # No collision detection; all particles move freely
            move_flags[:] = True  # All particles can move
            collided[:] = False   # No particles have collided

        # Update positions and orientations
        positions = np.where(move_flags[:, np.newaxis], desired_positions, positions)
        orientations = new_orientations  # Orientation always updated

        if render_animation:
            # Drawing
            for i in range(num_particles):
                # Draw the particle as a circle with an arrow indicating orientation
                circle = plt.Circle(positions[i], particle_diameter / 2, color='k', fill=False)
                ax.add_artist(circle)
                # Arrow
                arrow_length = particle_diameter / 2
                dx = arrow_length * np.cos(orientations[i])
                dy = arrow_length * np.sin(orientations[i])
                color = 'grey' if not collided[i] else 'red'
                ax.arrow(positions[i][0], positions[i][1], dx, dy, head_width=0.005, head_length=0.01,
                         fc=color, ec=color, width=0.001)
            return []

    if render_animation:
        # Create animation using FuncAnimation
        ani = animation.FuncAnimation(fig, animate, frames=max_steps, init_func=init, blit=False, interval=50)
        # Display the animation
        display(HTML(ani.to_html5_video()))
        plt.close(fig)
    else:
        # Run the simulation without rendering
        for step in range(max_steps):
            animate(step)

    print("Completed.")

    return positions, orientations


In [2]:
def trace(data):
  x_labels = list(data.keys())
  x = []
  y = []
  for i, label in enumerate(x_labels):
      for value in data[label]:
          x.append(i)
          y.append(value)
  plt.figure(figsize=(4, 3))
  plt.scatter(x, y, marker='x', color='blue')
  plt.xticks(ticks=range(len(x_labels)), labels=x_labels)
  plt.xlabel("d")
  plt.ylabel("p")
  plt.title("polarisation vs. density")
  plt.show()
  return

# Exemple de code pour tracer vos stats
# valeurs arbitraires pour l'exemple, stoquées dans un dictionnaire
# clé: densité
# valeurs: polarisation pour chaque run (ex: 4 valeurs, 4 runs)
#data = {
#  "0.1" : [0.01, 0.03, 0.021],
#  "0.2" : [0.02],
#  "0.5" : [0.81, 0.93],
#  "0.7" : [0.88, 0.83],
#  "0.99" : [0.11, 0.23, 0.18, 0.12]
#}
#trace(data)


---
---
---

#<font color="orange">Prise en main: lancer une simulation</font>

La fonction *run_simulation* permet de générer une simulation, qui est ensuite affichée. En l'absence de valeurs explicites, les valeurs de paramètre par défaut sont utilisées.

Voici ci-dessous comment appeler la fonction de manière explicite, avec les valeurs par défaut (i.e. équivalent à lancer ```run_simulation()```):

```
run_simulation(
    num_particles=64,
    max_steps=200,
    particle_diameter=0.02,
    sensing_radius=8 * 0.02,  # 8 * particle_diameter
    repulsion_threshold=0.3,
    attraction_threshold=0.8,
    speed=0.01,
    rotation_speed=np.pi / 8,
    box_size=1.0,
    rotational_noise_std=0.05,
    enable_collision=True,
    render_animation=True
)
```

Il est possible de lancer une simulation en ne donnant qu'un seul paramètre (ou quelques uns). Par exemple, l'appel suivant lance une simulation avec 1024 robots et n'affiche pas de vidéo (cela permet d'accélérer très significativement le calcul).

```positions, orientations = run_simulation(num_particles=1024, render_animation=False)```

Cette fonction renvoie deux tableaux, contenant pour la dernière itération de la simulation (1) la position de chaque agent et (2) l'orientation de chaque agent (-PI,+PI).

Exécutez la cellule suivante. Il s'agit de ce que nous considèrons ici comme dynamique de référence. Comme vous pourrez le voir, la génération de la vidéo prend une 30aine de seconde.

In [3]:
positions, orientations = run_simulation()

Simulation Parameters:
Number of particles: 64
Max steps: 200
Particle diameter: 0.02
Sensing radius: 0.16
Repulsion threshold (normalized): 0.3
Attraction threshold (normalized): 0.8
Max speed: 0.01
Max rotation speed: 0.39
Rotational noise std: 0.05
Collision enabled: True
Box size: 1.0
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% [done].


Completed.


Exécutez la cellule ci-dessous. La simulation est exactement la même, mais il n'y a pas d'affichage. Comme vous pouvez le voir, l'exécution est *beaucoup* plus rapide. Ce sera utile si vous devez générer rapidement beaucoup de données.

In [4]:
positions, orientations = run_simulation(render_animation=False)

Simulation Parameters:
Number of particles: 64
Max steps: 200
Particle diameter: 0.02
Sensing radius: 0.16
Repulsion threshold (normalized): 0.3
Attraction threshold (normalized): 0.8
Max speed: 0.01
Max rotation speed: 0.39
Rotational noise std: 0.05
Collision enabled: True
Box size: 1.0
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% [done].
Completed.


---
---
---

#<font color="orange">Exercice 1 : mesure de la cohésion de l'essaim</font>

Etudiez et lancez le code ci-dessous. Vous devez ensuite étendre le code pour calculer une mesure de cohésion sur l'orientation des agents. L'essaim est maximalement polarisé lorsque tous les individus vont dans la même direction.

Vous pouvez utiliser la formule suivante pour calculer la polarisation P, en considérant chaque agent comme un vecteur:

```P = 1/N x | Sum of all vectors |```

Si P = 1, l'ensemble des individus de l'essaim vont dans la même direction (on parlera de particules alignées). Si P=0, l'essaim est maximalement désordonné.

Une autre méthode consiste à calculer l'écart type entre l'ensemble des orientations. A vous de décider votre méthode.

Aide: *attention lorsque vous effectuez des calculs en utilisant la valeur d'orientation: souvenez-vous que des valeurs approchant -PI ou +PI sont très proches.*

In [7]:
positions, orientations = run_simulation(render_animation=True)

sum_v = 0

# Display final state
for i in range(len(orientations)):
    print(f"Particle {i} at ({positions[i][0]:.2f},{positions[i][1]:.2f}) with orientation {orientations[i]:.2f}")
    sum_v += orientations[i]

Simulation Parameters:
Number of particles: 64
Max steps: 200
Particle diameter: 0.02
Sensing radius: 0.16
Repulsion threshold (normalized): 0.3
Attraction threshold (normalized): 0.8
Max speed: 0.01
Max rotation speed: 0.39
Rotational noise std: 0.05
Collision enabled: True
Box size: 1.0
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% [done].


Completed.
Particle 0 at (0.20,0.74) with orientation 2.83
Particle 1 at (0.35,0.89) with orientation 2.78
Particle 2 at (0.48,0.81) with orientation 2.77
Particle 3 at (0.75,0.58) with orientation 2.75
Particle 4 at (0.35,0.26) with orientation 2.75
Particle 5 at (0.09,1.00) with orientation 2.78
Particle 6 at (0.41,0.94) with orientation 2.75
Particle 7 at (0.49,0.11) with orientation 2.74
Particle 8 at (0.92,0.23) with orientation 2.77
Particle 9 at (0.30,0.37) with orientation 2.77
Particle 10 at (0.84,0.26) with orientation 2.82
Particle 11 at (0.41,0.81) with orientation 2.81
Particle 12 at (0.88,0.07) with orientation 2.77
Particle 13 at (0.88,0.90) with orientation 2.78
Particle 14 at (0.21,0.22) with orientation 2.77
Particle 15 at (0.14,0.15) with orientation 2.64
Particle 16 at (0.41,0.21) with orientation 2.89
Particle 17 at (0.27,0.95) with orientation 2.82
Particle 18 at (0.15,0.76) with orientation 2.82
Particle 19 at (0.19,0.92) with orientation 2.84
Particle 20 at (0.6

In [8]:
P = (1/len(orientations)) * (abs(sum_v))
print("Measure of cohesion: ", P)

Measure of cohesion:  2.781789087587127


---
---
---

#<font color="orange">Exercice 2 : transition de phase en fonction de la densité</font>

La **densité** est un paramètre qui a un rôle important sur la dynamique d'un essaim. La densité est maximale (d=1) lorsqu'il n'est plus possible d'ajouter d'agents dans l'environnement. On peut la calculer comme le rapport entre le nombre d'individus présents et le nombre maximum d'individus que l'environnement peut contenir.

Ici, on utilisera un calcul approché de la densité en faisant le rapport entre la surface totale occupée par l'essaim ```N x ( PI x particle_radius^2 )``` et la surface totale de l'environnement ```box_size x box_size```. C'est un peu faux (pourquoi? est-ce que d=1 est possible?) mais largement suffisant ici.

Etendez le code présent dans la cellule ci-dessous pour simuler un essaim de Boids, en étudiant successivement plusieurs niveaux de densité. L'objectif est de voir si la polarisation change selon la densité. Pour cela, vous tracerez un graphe avec la valeur de la densité en X et la polarisation à l'état terminal en Y. Pour chaque valeur de X, vous pouvez tracer le résultat de plusieurs runs indépendants (il s'agit d'un modèle stochastique).

Conseil: *pour simplifier, vous pouvez vous limiter à faire varier la taille de l'environnement, en conservant le même nombre d'agents.*


In [ ]:
# densité

# Exemple de code pour tracer vos stats
# valeurs arbitraires pour l'exemple, stoquées dans un dictionnaire
# clé: densité
# valeurs: polarisation pour chaque run (ex: 4 valeurs, 4 runs)
data = {
  "0.1" : [0.01, 0.03, 0.021],
  "0.2" : [0.02],
  "0.5" : [0.81, 0.93],
  "0.7" : [0.88, 0.83],
  "0.99" : [0.11, 0.23, 0.18, 0.12]
}
trace(data)


---
---
---

#<font color="orange">Exercice 3 : sensibilité aux paramètres de contrôle</font>

Outre la densité, voici d'autres paramètres qui influent significativement la dynamique de l'essaim:
- la **distance d'interaction**, qui dépend de la distance à laquelle deux agents peuvent se voir (ie. voisins immédiats). Voir le paramètre ```sensing_radius```.
- les **seuils** de déclenchement des règles de comportements (un seuil de proximité pour la règle de répulsion, un seuil d'éloignement pour la règle d'attraction).  Voir les paramètres ```repulsion_threshold``` et ```attraction_threshold```.
- l'existence (et, le cas échéant, l'amplitude) d'une **perturbation** sur la direction de chaque agent. Voir le paramètre ```rotational_noise_std```.

Pour chacune des familles de paramètres cités ci-avant, menez un étude de la dynamique de l'essaim. Vous devez générer la vidéo d'un cas typique *et* tracer une figure représentant la dynamique (phase ordonnée, phase désordonnée) selon les valeurs de paramètres.

Suggestions:
- vérifiez que votre simulation a le temps de converger,
- faîtes plusieurs runs pour un jeu de paramètres,
- testez des valeurs extrèmes (minimum **et** maximum).

In [ ]:
# distance d'interaction

In [ ]:
# seuils de repulsion, attraction, orientation

In [ ]:
# perturbation de l'orientation

Evaluez l'impact de la présence (ou non) d'**interactions physiques** de type corps dur entre les robots (i.e. collision possible ou non). Voir le paramètre ```enable_collision```. Pour cela, vous comparerez deux simulations avec une densité évelée (ex.: 64 robots, box size < 0.5) avec et sans collision.  Générez une vidéo pour chacune des simulations.

In [ ]:
# influence de la collision

---
---
---

#<font color="orange">Exercice 4 : dynamique singulière</font>

Vous avez jusqu'ici observé des comportements desordonnés ou alignés. Tentez maintenant d'obtenir un comportement collectif en anneau (ou donuts, ou tore). Vous pouvez désactiver la physique des agents pour cet exercice. Une vidéo suffit.

In [ ]:
# dynamique d'essaim en tore. Bon courage!

---
---
---

#<font color="orange">Exercice 5 : MSD - Mean Square Displacement</font>

Vous êtes en avance, bravo. Tracez le déplacement quadratique moyen des particules pour diverses simulations.

Aide: https://en.wikipedia.org/wiki/Mean_squared_displacement
